In [1]:
# Dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base

# Assign declarative base class
Base = declarative_base()

# Define Table Classes

In [2]:
# Define Measurement class
class Measurement(Base):
    __tablename__ = "measurements"
    id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(String)
    prcp = Column(Float)
    tobs = Column(Integer)
    
# Define Station class
class Station(Base):
    __tablename__ = "stations"
    id = Column(Integer, primary_key=True)
    station = Column(String)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)

In [3]:
# Create engine to database file
engine = create_engine("sqlite:///database/hawaii.sqlite")

# Create measurements and stations tables in database
Base.metadata.create_all(engine)

# Prepare CSV Data

In [4]:
# Csv file paths
clean_measure_path = "clean_data/clean_hawaii_measurements.csv"
clean_station_path = "clean_data/clean_hawaii_stations.csv"

# Load csv into pandas dfs
clean_measure_df = pd.read_csv(clean_measure_path, encoding="utf-8")
clean_station_df = pd.read_csv(clean_station_path, encoding="utf-8")

In [5]:
# Orient clean_measure_df
measurements_data = clean_measure_df.to_dict(orient='records')
measurements_data[:5]

[{'date': '2010-01-01', 'prcp': 0.08, 'station': 'USC00519397', 'tobs': 65},
 {'date': '2010-01-02', 'prcp': 0.0, 'station': 'USC00519397', 'tobs': 63},
 {'date': '2010-01-03', 'prcp': 0.0, 'station': 'USC00519397', 'tobs': 74},
 {'date': '2010-01-04', 'prcp': 0.0, 'station': 'USC00519397', 'tobs': 76},
 {'date': '2010-01-07', 'prcp': 0.06, 'station': 'USC00519397', 'tobs': 70}]

In [6]:
# Orient clean_station_df
stations_data = clean_station_df.to_dict(orient='records')
stations_data[0]

{'elevation': 3.0,
 'latitude': 21.2716,
 'longitude': -157.8168,
 'name': 'WAIKIKI 717.2, HI US',
 'station': 'USC00519397'}

# Insert Data Into Tables

In [7]:
# Use MetaData to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [8]:
# Save reference to the `measuerments` and `stations` tables, assign to variables
measurements = sqlalchemy.Table('measurements', metadata, autoload=True)
stations = sqlalchemy.Table('stations', metadata, autoload=True)

In [9]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [10]:
# Remove any pre-existing data.
conn.execute(measurements.delete())
conn.execute(stations.delete())

In [11]:
# Insert the data into tables
conn.execute(measurements.insert(), measurements_data)
conn.execute(stations.insert(), stations_data)

# Check Insert

In [12]:
# Fetch first five rows of measurements table 
conn.execute("select * from measurements limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0, 63),
 (3, 'USC00519397', '2010-01-03', 0, 74),
 (4, 'USC00519397', '2010-01-04', 0, 76),
 (5, 'USC00519397', '2010-01-07', 0.06, 70)]

In [13]:
# Fetch first five rows of stations table 
conn.execute("select * from stations limit 5").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]